In [138]:
import pandas as pd


In [139]:
# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize, sent_tokenize
# # stopwords: pronoun, wh-words. Maybe we should not remove these words


# stemmer = nltk.stem.SnowballStemmer('english')
# lemmatizer = nltk.wordnet.WordNetLemmatizer()

# list1 = raw_data.question1.values
# for q in raw_data.question1.values:
#     try:
#         word_tokenize(q)
#     except:
#         print(q)
#         break
        
# tokens_q1 = [word_tokenize(q) for q in raw_data.question1.values]
# tokens_q2 = [word_tokenize(q) for q in raw_data.question2.values]

In [274]:
# tf-idf
import nltk
from collections import defaultdict
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

translate_table = dict((ord(char), None) for char in string.punctuation)  
def tokenize(text):
    stemmer = PorterStemmer()
    def stem_tokens(tokens, stemmer):
        stemmed = []
        for item in tokens:
            stemmed.append(stemmer.stem(item))
        return stemmed
    
    lowers = text.lower()
    no_punctuation = lowers.translate(translate_table)
    tokens = nltk.word_tokenize(no_punctuation)
    stems = stem_tokens(tokens, stemmer)
    return stems

def process(corpus, max_features = 100):
#     porterstem = PorterStemmer()
#     porter_token = lambda text:tokenize(text, porterstem)
#     tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
    tfidf = TfidfVectorizer(tokenizer=tokenize, max_features = 100)
    tfs = tfidf.fit(corpus)
    return tfidf
def tf_idf_vector(q, tfidf):
    return pd.DataFrame(tfidf.transform([q]).toarray())

def tf_idf_similarity(q1, q2, tfidf):
    m = tfidf.transform([q1, q2])
    return cosine_similarity(m[0], m[1])

def feature_eng_tfidf(df, tfidf = None):
    max_features = 100
    corpus = list(qpair['question1']) + list(qpair['question2'])
    if tfidf is None:
        tfidf = process(corpus, max_features = max_features)
#     getvec = lambda q: tf_idf_vector(q, tfidf)
    
#     q1tfidf = df['question1'].apply(getvec)
    q1tfidf = pd.DataFrame()
    
#     q2tfidf = df['question2'].apply(getvec)
    q2tfidf = pd.DataFrame()
    for idx, r in df.iterrows():
        v1 = tf_idf_vector(r['question1'], tfidf)
        v2 = tf_idf_vector(r['question2'], tfidf)
        q1tfidf = q1tfidf.append(v1, ignore_index=True)
        q2tfidf = q2tfidf.append(v2, ignore_index=True)
    q1tfidf.columns = ["q1tfidx_"+str(i) for i in range(max_features)]
    q2tfidf.columns = ["q2tfidx_"+str(i) for i in range(max_features)]
    
    q1tfidf.index = df.index
    q2tfidf.index = df.index
    df.reset_index()
    
    df = pd.concat([df, q1tfidf, q2tfidf], axis=1)
    return df
    

In [141]:
import pandas as pd
import pickle
import nltk
nltk.download('punkt')
qpair = pd.read_csv("../data/qpairs_dev.csv")
qpair = qpair.dropna()
corpus = list(qpair['question1']) + list(qpair['question2'])
tfidf = process(corpus, 1000)


# for i in range(50):
#     q1 = qpair.loc[i]['question1']
#     q2 = qpair.loc[0]['question2']
#     print(tf_idf_vector(q1,tfidf))
#     print(tf_idf_similarity(q1, q2, tfidf))
    

[nltk_data] Downloading package punkt to /Users/wenyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [277]:
qpair = feature_eng(qpair)
tfidfqpair = feature_eng_tfidf(qpair,tfidf)


   id  qid1  qid2                                          question1  \
0   0     1     2  What is the step by step guide to invest in sh...   
1   1     3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   
2   3     7     8  Why am I mentally very lonely? How can I solve...   

                                           question2  is_duplicate  q1tfidx_0  \
0  What is the step by step guide to invest in sh...             0        0.0   
1  What would happen if the Indian government sto...             0        0.0   
2  Find the remainder when [math]23^{24}[/math] i...             0        0.0   

   q1tfidx_1  q1tfidx_2  q1tfidx_3     ...      q2tfidx_90  q2tfidx_91  \
0        0.0        0.0        0.0     ...             0.0         0.0   
1        0.0        0.0        0.0     ...             0.0         0.0   
2        0.0        0.0        0.0     ...             0.0         0.0   

   q2tfidx_92  q2tfidx_93  q2tfidx_94  q2tfidx_95  q2tfidx_96  q2tfidx_97  \
0         0.

In [273]:
qshort = qpair[0:3]
print(qshort.columns)

print(tf_idf_vector(corpus[0], tfidf).shape)
q1tfidf = pd.DataFrame()
for idx, r in qshort.iterrows():
    v1 = tf_idf_vector(r['question1'], tfidf)
#     print(v1)
    q1tfidf=q1tfidf.append(v1)
#     print(q1tfidf)
q1tfidf.columns = ["q1tfidx_"+str(i) for i in range(100)]
q1tfidf.index=qshort.index
# qshort.reset_index(inplace=True)
print(q1tfidf.shape, qshort.shape)
print(q1tfidf.index)
cc = pd.concat([q1tfidf, qshort],axis=1)
print(cc)

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], dtype='object')
(1, 100)
(3, 100) (3, 6)
Int64Index([0, 1, 2], dtype='int64')
   q1tfidx_0  q1tfidx_1  q1tfidx_2  q1tfidx_3  q1tfidx_4  q1tfidx_5  \
0        0.0        0.0        0.0        0.0   0.000000        0.0   
1        0.0        0.0        0.0        0.0   0.000000        0.0   
2        0.0        0.0        0.0        0.0   0.552623        0.0   

   q1tfidx_6  q1tfidx_7  q1tfidx_8  q1tfidx_9      ...       q1tfidx_96  \
0        0.0        0.0        0.0        0.0      ...              0.0   
1        0.0        0.0        0.0        0.0      ...              0.0   
2        0.0        0.0        0.0        0.0      ...              0.0   

   q1tfidx_97  q1tfidx_98  q1tfidx_99  id  qid1  qid2  \
0         0.0         0.0         0.0   0     1     2   
1         0.0         0.0         0.0   1     3     4   
2         0.0         0.0         0.0   3     7     8   

                                   

In [ ]:

# q1tfidf = pd.DataFrame(tf_idf_vector(corpus[0], tfidf).toarray())
# q1tfidf.columns = ["q1tfidf_"+str(i) for i in range(100)]
getvec = lambda q: tf_idf_vector(q, tfidf)
    

In [ ]:
# # pos_tag 
# import nltk
# nltk.download('wordnet')
# from nltk.corpus import wordnet as wn
# nltk.download('averaged_perceptron_tagger')

# def pos_synnet_similarity(q1, q2):
#     q1 = nltk.word_tokenize(q1)
#     q2 = tokenize(q2)
#     tag1 = nltk.pos_tag(q1)
#     tag2 = nltk.pos_tag(q2)
    
#     print(tag1, tag2)


    
# print(corpus[1], corpus[2])
# pos_synnet_similarity(corpus[1], corpus[2])


In [97]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import re

_WORD_SPLIT = re.compile("([.,!?\"':;)(])")
UNI_BLEU_WEIGHTS = (1, 0, 0, 0)
BI_BLEU_WEIGHTS = (0, 1, 0, 0)
BLEU2_WEIGHTS = (0.5, 0.5, 0, 0)


# def save_results(predictions, IDs, filename):
#     with open(filename, 'w') as f:
#         f.write("test_id,is_duplicate\n")
#         for i, pred in enumerate(predictions):
#             f.write("%d,%f\n" % (IDs[i], pred))


def tokenizer(sentence):
    """Very basic tokenizer: split the sentence by space into a list of tokens."""
    words = []
    for space_separated_fragment in sentence.strip().split():
        words.extend(re.split(_WORD_SPLIT, space_separated_fragment))
    return [w for w in words if w]



def char_ngram_tokenizer(sentence, n):
    """Character ngram tokenizer: split the sentence into a list of char ngram tokens."""
    return [sentence[i:i+n] for i in range(len(sentence)-n+1)]


def get_word_count(x):
    return len(tokenizer(str(x)))

def word_overlap(x):
    return len(set(str(x['question1']).lower().split()).intersection(
             set(str(x['question2']).lower().split())))


def char_bigram_overlap(x):
    return len(set(char_ngram_tokenizer(str(x['question1']), 2)).intersection(
             set(char_ngram_tokenizer(str(x['question2']), 2))))


def char_trigram_overlap(x):
    return len(set(char_ngram_tokenizer(str(x['question1']), 3)).intersection(
             set(char_ngram_tokenizer(str(x['question2']), 3))))


def char_fourgram_overlap(x):
    return len(set(char_ngram_tokenizer(str(x['question1']), 4)).intersection(
             set(char_ngram_tokenizer(str(x['question2']), 4))))


def get_uni_BLEU(x):
    s_function = SmoothingFunction()
    # method 2 is add 1 smoothing
    return sentence_bleu([tokenizer(str(x['question2']))],
                         tokenizer(str(x['question1'])),
                         weights=UNI_BLEU_WEIGHTS,
                         smoothing_function=s_function.method2)


def get_bi_BLEU(x):
    s_function = SmoothingFunction()
    # method 2 is add 1 smoothing
    return sentence_bleu([tokenizer(str(x['question2']))],
                         tokenizer(str(x['question1'])),
                         weights=BI_BLEU_WEIGHTS,
                         smoothing_function=s_function.method2)


def get_BLEU2(x):
    s_function = SmoothingFunction()
    # method 2 is add 1 smoothing
    return sentence_bleu([tokenizer(str(x['question2']))],
                         tokenizer(str(x['question1'])),
                         weights=BLEU2_WEIGHTS,
                         smoothing_function=s_function.method2)


def feature_eng(df):

    # word count of question 1
    df['q1_word_count'] = df['question1'].apply(get_word_count)

    # word count of question 2
    df['q2_word_count'] = df['question2'].apply(get_word_count)

    # word count difference
    df['word_count_diff'] = abs(df['q1_word_count'] - df['q2_word_count'])

    # number of word overlap between q1 and q2
    df['word_overlap'] = df.apply(word_overlap, axis=1)

    # unigram BLEU score
    df['uni_BLEU'] = df.apply(get_uni_BLEU, axis=1)

    # bigram BLEU score
    df['bi_BLEU'] = df.apply(get_bi_BLEU, axis=1)

    # BLEU2 score
    df['BLEU2'] = df.apply(get_BLEU2, axis=1)

    # character unigram overlap
    df['char_bigram_overlap'] = df.apply(char_bigram_overlap, axis=1)

    # character trigram overlap
    df['char_trigram_overlap'] = df.apply(char_trigram_overlap, axis=1)

    # character 4-gram overlap
    df['char_4gram_overlap'] = df.apply(char_fourgram_overlap, axis=1)

    return df

In [322]:
# Synset similarity 
import nltk
nltk.download('brown')
from nltk.corpus import wordnet as wn
from nltk.corpus import brown
import math
import numpy as np
import sys

# Parameters to the algorithm. Currently set to values that was reported
# in the paper to produce "best" results.
ALPHA = 0.2
BETA = 0.45
ETA = 0.4
PHI = 0.2
DELTA = 0.85

brown_freqs = dict()
N = 0

######################### word similarity ##########################

def get_best_synset_pair(word_1, word_2):
    """ 
    Choose the pair with highest path similarity among all pairs. 
    Mimics pattern-seeking behavior of humans.
    """
    max_sim = -1.0
    synsets_1 = wn.synsets(word_1)
    synsets_2 = wn.synsets(word_2)
    if len(synsets_1) == 0 or len(synsets_2) == 0:
        return None, None
    else:
        max_sim = -1.0
        best_pair = None, None
        for synset_1 in synsets_1:
            for synset_2 in synsets_2:
                sim = wn.path_similarity(synset_1, synset_2)
#                 print(sim)
                if sim != None and sim > max_sim:
                    max_sim = sim
                    best_pair = synset_1, synset_2
        return best_pair

def length_dist(synset_1, synset_2):
    """
    Return a measure of the length of the shortest path in the semantic 
    ontology (Wordnet in our case as well as the paper's) between two 
    synsets.
    """
    l_dist = math.inf
    if synset_1 is None or synset_2 is None: 
        return 0.0
    if synset_1 == synset_2:
        # if synset_1 and synset_2 are the same synset return 0
        l_dist = 0.0
    else:
        wset_1 = set([str(x.name()) for x in synset_1.lemmas()])        
        wset_2 = set([str(x.name()) for x in synset_2.lemmas()])
        if len(wset_1.intersection(wset_2)) > 0:
            # if synset_1 != synset_2 but there is word overlap, return 1.0
            l_dist = 1.0
        else:
            # just compute the shortest path between the two
            l_dist = synset_1.shortest_path_distance(synset_2)
            if l_dist is None:
                l_dist = 0.0
    # normalize path length to the range [0,1]
    return math.exp(-ALPHA * l_dist)

def hierarchy_dist(synset_1, synset_2):
    """
    Return a measure of depth in the ontology to model the fact that 
    nodes closer to the root are broader and have less semantic similarity
    than nodes further away from the root.
    """
    h_dist = math.inf
    if synset_1 is None or synset_2 is None: 
        return h_dist
    if synset_1 == synset_2:
        # return the depth of one of synset_1 or synset_2
        h_dist = max([x[1] for x in synset_1.hypernym_distances()])
    else:
        # find the max depth of least common subsumer
        hypernyms_1 = {x[0]:x[1] for x in synset_1.hypernym_distances()}
        hypernyms_2 = {x[0]:x[1] for x in synset_2.hypernym_distances()}
        lcs_candidates = set(hypernyms_1.keys()).intersection(
            set(hypernyms_2.keys()))
        if len(lcs_candidates) > 0:
            lcs_dists = []
            for lcs_candidate in lcs_candidates:
                lcs_d1 = 0
                if lcs_candidate not in hypernyms_1:
                    lcs_d1 = hypernyms_1[lcs_candidate]
                lcs_d2 = 0
                if lcs_candidate not in hypernyms_2:
                    lcs_d2 = hypernyms_2[lcs_candidate]
                lcs_dists.append(max([lcs_d1, lcs_d2]))
            h_dist = max(lcs_dists)
        else:
            h_dist = 0
    return ((math.exp(BETA * h_dist) - math.exp(-BETA * h_dist)) / 
        (math.exp(BETA * h_dist) + math.exp(-BETA * h_dist)))
    
def word_similarity(word_1, word_2):
    synset_pair = get_best_synset_pair(word_1, word_2)
    return (length_dist(synset_pair[0], synset_pair[1]) * 
        hierarchy_dist(synset_pair[0], synset_pair[1]))

######################### sentence similarity ##########################

def most_similar_word(word, word_set):
    """
    Find the word in the joint word set that is most similar to the word
    passed in. We use the algorithm above to compute word similarity between
    the word and each word in the joint word set, and return the most similar
    word and the actual similarity value.
    """
    max_sim = -1.0
    sim_word = ""
    for ref_word in word_set:
        sim = word_similarity(word, ref_word)
        if sim > max_sim:
            max_sim = sim
            sim_word = ref_word
    return sim_word, max_sim
    
def info_content(lookup_word):
    """
    Uses the Brown corpus available in NLTK to calculate a Laplace
    smoothed frequency distribution of words, then uses this information
    to compute the information content of the lookup_word.
    """
    global N
    if N == 0:
        # poor man's lazy evaluation
        for sent in brown.sents():
            for word in sent:
                word = word.lower()
                if word not in brown_freqs:
                    brown_freqs[word] = 0
                brown_freqs[word] = brown_freqs[word] + 1
                N = N + 1
    lookup_word = lookup_word.lower()
    n = 0 if lookup_word not in brown_freqs else brown_freqs[lookup_word]
    return 1.0 - (math.log(n + 1) / math.log(N + 1))
    
def semantic_vector(words, joint_words, info_content_norm):
    """
    Computes the semantic vector of a sentence. The sentence is passed in as
    a collection of words. The size of the semantic vector is the same as the
    size of the joint word set. The elements are 1 if a word in the sentence
    already exists in the joint word set, or the similarity of the word to the
    most similar word in the joint word set if it doesn't. Both values are 
    further normalized by the word's (and similar word's) information content
    if info_content_norm is True.
    """
    sent_set = set(words)
    semvec = np.zeros(len(joint_words))
    i = 0
    for joint_word in joint_words:
        if joint_word in sent_set:
            # if word in union exists in the sentence, s(i) = 1 (unnormalized)
            semvec[i] = 1.0
            if info_content_norm:
                semvec[i] = semvec[i] * math.pow(info_content(joint_word), 2)
        else:
            # find the most similar word in the joint set and set the sim value
            sim_word, max_sim = most_similar_word(joint_word, sent_set)
            semvec[i] = PHI if max_sim > PHI else 0.0
            if info_content_norm:
                semvec[i] = semvec[i] * info_content(joint_word) * info_content(sim_word)
        i = i + 1
    return semvec                
            
def semantic_similarity(sentence_1, sentence_2, info_content_norm):
    """
    Computes the semantic similarity between two sentences as the cosine
    similarity between the semantic vectors computed for each sentence.
    """
    words_1 = nltk.word_tokenize(sentence_1)
    words_2 = nltk.word_tokenize(sentence_2)
    joint_words = set(words_1).union(set(words_2))
    vec_1 = semantic_vector(words_1, joint_words, info_content_norm)
    vec_2 = semantic_vector(words_2, joint_words, info_content_norm)
    return np.dot(vec_1, vec_2.T) / (np.linalg.norm(vec_1) * np.linalg.norm(vec_2))

######################### word order similarity ##########################

def word_order_vector(words, joint_words, windex):
    """
    Computes the word order vector for a sentence. The sentence is passed
    in as a collection of words. The size of the word order vector is the
    same as the size of the joint word set. The elements of the word order
    vector are the position mapping (from the windex dictionary) of the 
    word in the joint set if the word exists in the sentence. If the word
    does not exist in the sentence, then the value of the element is the 
    position of the most similar word in the sentence as long as the similarity
    is above the threshold ETA.
    """
    wovec = np.zeros(len(joint_words))
    i = 0
    wordset = set(words)
    for joint_word in joint_words:
        if joint_word in wordset:
            # word in joint_words found in sentence, just populate the index
            wovec[i] = windex[joint_word]
        else:
            # word not in joint_words, find most similar word and populate
            # word_vector with the thresholded similarity
            sim_word, max_sim = most_similar_word(joint_word, wordset)
            if max_sim > ETA:
                wovec[i] = windex[sim_word]
            else:
                wovec[i] = 0
        i = i + 1
    return wovec

def word_order_similarity(sentence_1, sentence_2):
    """
    Computes the word-order similarity between two sentences as the normalized
    difference of word order between the two sentences.
    """
    words_1 = nltk.word_tokenize(sentence_1)
    words_2 = nltk.word_tokenize(sentence_2)
    joint_words = list(set(words_1).union(set(words_2)))
    windex = {x[1]: x[0] for x in enumerate(joint_words)}
    r1 = word_order_vector(words_1, joint_words, windex)
    r2 = word_order_vector(words_2, joint_words, windex)
    return 1.0 - (np.linalg.norm(r1 - r2) / np.linalg.norm(r1 + r2))

######################### overall similarity ##########################

def similarity(sentence_1, sentence_2, info_content_norm):
    """
    Calculate the semantic similarity between two sentences. The last 
    parameter is True or False depending on whether information content
    normalization is desired or not.
    """
    return DELTA * semantic_similarity(sentence_1, sentence_2, info_content_norm) + \
        (1.0 - DELTA) * word_order_similarity(sentence_1, sentence_2)
def feature_eng_semantic(df):
    sim = []
    for idx, r in df.iterrows():
        sim.append(similarity(r['question1'], r['question2'], True))
    df['sem_sim'] = sim
    return df

[nltk_data] Downloading package brown to /Users/wenyan/nltk_data...
[nltk_data]   Package brown is already up-to-date!


   id  qid1  qid2                                          question1  \
0   0     1     2  What is the step by step guide to invest in sh...   
1   1     3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   

                                           question2  is_duplicate   sem_sim  
0  What is the step by step guide to invest in sh...             0  0.938124  
1  What would happen if the Indian government sto...             0  0.670906  


/usr/local/anaconda3/envs/defaultenv/lib/python3.7/site-packages/ipykernel_launcher.py:245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [340]:
import time
print("begin:", time.time())
qpair1 = feature_eng(qpair)
print("qp2:", time.time())
# qpair2 = feature_eng_semantic(qpair1)
qpair2 = feature_eng_tfidf(qpair1)
print("done:", time.time())

begin: 1544329427.808866
qp2: 1544329674.245862


KeyboardInterrupt: 